<a href="https://colab.research.google.com/github/Dhiakacem/dk-Angular-Dashboard/blob/main/PlantdiseaseTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle


In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dhiakacem","key":"edd0b62b503dc4e0a4db5a4de33db32c"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [4]:
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

100% 2.70G/2.70G [02:07<00:00, 23.7MB/s]
100% 2.70G/2.70G [02:07<00:00, 22.7MB/s]


In [6]:

import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam


In [7]:
with zipfile.ZipFile('new-plant-diseases-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('new_plant_diseases_dataset')


In [8]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'new_plant_diseases_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
validation_dir = 'new_plant_diseases_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
extracted_dir = 'new_plant_diseases_dataset/New Plant Diseases Dataset(Augmented)'
print(os.listdir(extracted_dir))

if not os.path.exists(train_dir):
    print("Training directory not found:", train_dir)
elif not os.path.exists(validation_dir):
    print("Validation directory not found:", validation_dir)
else:
    print("Directories exist. Continuing with data generation and model training...")

    # Continue with the rest of your code for data generation and model training


['New Plant Diseases Dataset(Augmented)']
Directories exist. Continuing with data generation and model training...


In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Check if the directories exist
if not os.path.exists(train_dir):
    print("Training directory not found.")
elif not os.path.exists(validation_dir):
    print("Validation directory not found.")
else:
    # Flow training images in batches of 32 using train_datagen generator
    train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        class_mode='categorical')

    # Flow validation images in batches of 32 using validation_datagen generator
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,  # This is the source directory for validation images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        class_mode='categorical')

    # Define the model architecture and compile the model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dense(38, activation='softmax')  # Adjust the number of classes as needed
    ])

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size
    )


Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
2196/2196 [==============================] - 512s 232ms/step - loss: 2.0392 - accuracy: 0.4125 - val_loss: 1.4897 - val_accuracy: 0.5701


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
model.save('models.h5')


In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

app = Flask(__name__)

# Load the saved model
model = load_model('models.h5')

# Define function to preprocess image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize pixel values
    return img

@app.route('/predict', methods=['POST' , 'GET'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        # Save the uploaded image
        file_path = '/tmp/uploaded_image.jpg'
        file.save(file_path)

        # Preprocess the image
        img = preprocess_image(file_path)

        # Make prediction
        prediction = model.predict(img)

        # Process prediction results
        # For simplicity, assuming binary classification (disease vs. no disease)
        # and returning a boolean indicating disease presence
        is_disease = bool(prediction[0][0] > 0.5)

        if is_disease:
            # Add logic to classify the type of disease if present
            # You may need additional processing and post-processing steps here
            # For now, returning a generic response
            return jsonify({'is_disease': is_disease, 'disease_type': 'Unknown'})
        else:
            return jsonify({'is_disease': is_disease, 'disease_type': None})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
